In [7]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST, FashionMNIST
from torch.utils.data import DataLoader
import lightning as L

In [2]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))

    def forward(self, x):
        return self.l1(x)


class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

    def forward(self, x):
        return self.l1(x)

In [150]:
class LitAutoEncoder(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(64, 1, 28, 28)
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
        self.decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.encoder(x)
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        return z
        
    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        val_loss = F.mse_loss(x_hat, x)
        self.log("val_loss", val_loss, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [134]:

dataset = FashionMNIST(os.getcwd(), download=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset, batch_size=64, num_workers=19,persistent_workers=True,
                          shuffle=True)
val_loader = DataLoader(dataset, len(dataset), num_workers=19, persistent_workers=True,
                          shuffle=False)

In [138]:
from lightning.pytorch.callbacks import DeviceStatsMonitor
# model
autoencoder = LitAutoEncoder()
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
early_stop_callback = EarlyStopping(monitor="train_loss", 
                                    min_delta=0.00, 
                                    patience=0, 
                                    verbose=True, 
                                    mode="min")

# train model
trainer = L.Trainer(max_epochs=2, default_root_dir=os.path.join(os.getcwd(), 'lightning_ckpts'),
                    callbacks=[])
trainer.fit(model=autoencoder, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params | In sizes  | Out sizes
---------------------------------------------------------------
0 | encoder | Sequential | 50.4 K | [64, 784] | [64, 3]  
1 | decoder | Sequential | 51.2 K | ?         | ?        
---------------------------------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [136]:
%reload_ext tensorboard
%tensorboard --logdir=./lightning_ckpts/lightning_logs/version_0/

Reusing TensorBoard on port 6020 (pid 40864), started 0:06:02 ago. (Use '!kill 40864' to kill it.)

In [151]:
model = LitAutoEncoder.load_from_checkpoint(os.path.join(os.getcwd(), 
                                        'lightning_ckpts', 'lightning_logs',
                                        'version_4', 'checkpoints', 
                                        'epoch=1-step=1876.ckpt'))
# model.eval()
# model(torch.rand(1, 28, 28, device=model.device))
trainer = L.Trainer()
predictions = trainer.predict(model, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [155]:
predictions[0].shape

torch.Size([60000, 3])